# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

In [2]:
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format

Импортировал необходимые для исследования библиотеки.

In [3]:
clients_data = pd.read_csv('/datasets/data.csv')

Прочитал файл data.csv из папки /datasets и сохранил его в переменной clients_data.
Сразу вывел первые 15 строк из датафрейма

In [4]:
clients_data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,"-8,437.67",42,высшее,0,женат / замужем,0,F,сотрудник,0,"253,875.64",покупка жилья
1,1,"-4,024.80",36,среднее,1,женат / замужем,0,F,сотрудник,0,"112,080.01",приобретение автомобиля
2,0,"-5,623.42",33,Среднее,1,женат / замужем,0,M,сотрудник,0,"145,885.95",покупка жилья
3,3,"-4,124.75",32,среднее,1,женат / замужем,0,M,сотрудник,0,"267,628.55",дополнительное образование
4,0,"340,266.07",53,среднее,1,гражданский брак,1,F,пенсионер,0,"158,616.08",сыграть свадьбу


Сразу видим некорректные значения в столбцах и пропущенные значения.

Вывел общую информацию по таблице

In [5]:
clients_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

В данных из таблицы нужно будет удалить дупликаты, заполнить пропуски, перевести отрицательные значения в положительные. В столбцах `days_employed` и `total_income` присутсвуют пропущенные значения.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [6]:
clients_data['days_employed'] = clients_data['days_employed'].apply(abs)
clients_data.loc[clients_data['days_employed'] > 30000, 'days_employed'] = clients_data.loc[clients_data['days_employed'] > 30000, 'days_employed'] /24

Перевёл отрицательные значения стажа работы к положительным значениям. Аномально большие значения перевел из часов в дни.

In [7]:
total_income_median = clients_data['total_income'].median()
days_employed_mean = clients_data['days_employed'].mean()

clients_data['days_employed'] = clients_data['days_employed'].fillna(days_employed_mean)
clients_data['total_income'] = clients_data['total_income'].fillna(total_income_median)

В `days_employed` пропуски заменили на среднее значение. В `total_income` пропуски заменили на медианное значение.

In [8]:
clients_data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,"8,437.67",42,высшее,0,женат / замужем,0,F,сотрудник,0,"253,875.64",покупка жилья
1,1,"4,024.80",36,среднее,1,женат / замужем,0,F,сотрудник,0,"112,080.01",приобретение автомобиля
2,0,"5,623.42",33,Среднее,1,женат / замужем,0,M,сотрудник,0,"145,885.95",покупка жилья
3,3,"4,124.75",32,среднее,1,женат / замужем,0,M,сотрудник,0,"267,628.55",дополнительное образование
4,0,"14,177.75",53,среднее,1,гражданский брак,1,F,пенсионер,0,"158,616.08",сыграть свадьбу


Убедились, что в таблице не осталось пропусков. Для этого посчитали пропущенные значения.

In [9]:
clients_data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [10]:
clients_data['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

В столбце gender увидел некорректные данные по признаку пола

In [11]:
clients_data[clients_data['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,"2,358.60",24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,"203,905.16",покупка недвижимости


Заменил значение

In [12]:
clients_data.loc[clients_data['gender'] == 'XNA', 'gender'] = 'M'
clients_data['gender'].unique()

array(['F', 'M'], dtype=object)

In [13]:
#создаем переменные по медиане зарплаты

import numpy as np

med_dob_years = clients_data.groupby('family_status')['dob_years'].transform('median')

clients_data['dob_years'] = clients_data['dob_years'].replace(0, np.nan)

#заменяем пропуски медианой по каждой группе
clients_data['dob_years'] = clients_data['dob_years'].fillna(med_dob_years)
clients_data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,"8,437.67",42.00,высшее,0,женат / замужем,0,F,сотрудник,0,"253,875.64",покупка жилья
1,1,"4,024.80",36.00,среднее,1,женат / замужем,0,F,сотрудник,0,"112,080.01",приобретение автомобиля
2,0,"5,623.42",33.00,Среднее,1,женат / замужем,0,M,сотрудник,0,"145,885.95",покупка жилья
3,3,"4,124.75",32.00,среднее,1,женат / замужем,0,M,сотрудник,0,"267,628.55",дополнительное образование
4,0,"14,177.75",53.00,среднее,1,гражданский брак,1,F,пенсионер,0,"158,616.08",сыграть свадьбу
5,0,926.19,27.00,высшее,0,гражданский брак,1,M,компаньон,0,"255,763.57",покупка жилья
6,0,"2,879.20",43.00,высшее,0,женат / замужем,0,F,компаньон,0,"240,525.97",операции с жильем
7,0,152.78,50.00,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,"135,823.93",образование
8,2,"6,929.87",35.00,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,"95,856.83",на проведение свадьбы
9,0,"2,188.76",41.00,среднее,1,женат / замужем,0,M,сотрудник,0,"144,425.94",покупка жилья для семьи


In [14]:
clients_data.dob_years.describe()

count   21,525.00
mean        43.49
std         12.22
min         19.00
25%         34.00
50%         43.00
75%         53.00
max         75.00
Name: dob_years, dtype: float64

**Вывод**

### Замена типа данных

In [15]:
clients_data['days_employed'] = clients_data['days_employed'].astype('int')
clients_data['total_income'] = clients_data['total_income'].astype('int')

In [16]:
clients_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  int64  
 2   dob_years         21525 non-null  float64
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int64  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


**Вывод**

В `days_employed` и `total_income` стояли значения типа float - заменили их на int.

### Обработка дубликатов

In [17]:
clients_data['education'] = clients_data['education'].str.lower()

In [18]:
clients_data.duplicated().sum()

print(f'Количество дубликатов по датафрейму: {clients_data.duplicated().sum()}')

Количество дубликатов по датафрейму: 72


In [19]:
clients_data[clients_data.duplicated(keep=False)].sort_values(by=['total_income', 'days_employed'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
120,0,4641,46.00,среднее,1,женат / замужем,0,F,сотрудник,0,145017,высшее образование
520,0,4641,35.00,среднее,1,гражданский брак,1,F,сотрудник,0,145017,сыграть свадьбу
541,0,4641,57.00,среднее,1,женат / замужем,0,F,сотрудник,0,145017,сделка с подержанным автомобилем
554,0,4641,60.00,среднее,1,женат / замужем,0,M,сотрудник,0,145017,покупка недвижимости
680,1,4641,30.00,высшее,0,женат / замужем,0,F,госслужащий,0,145017,покупка жилья для семьи
...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,4641,64.00,среднее,1,женат / замужем,0,F,пенсионер,0,145017,дополнительное образование
21032,0,4641,60.00,среднее,1,женат / замужем,0,F,пенсионер,0,145017,заняться образованием
21132,0,4641,47.00,среднее,1,женат / замужем,0,F,сотрудник,0,145017,ремонт жилью
21281,1,4641,30.00,высшее,0,женат / замужем,0,F,сотрудник,0,145017,покупка коммерческой недвижимости


In [20]:
clients_data = clients_data.drop_duplicates().reset_index(drop=True)

In [21]:
clients_data.duplicated().sum()

print(f'Количество дубликатов по датафрейму: {clients_data.duplicated().sum()}')

Количество дубликатов по датафрейму: 0


**Вывод**

Привели значения `education` к единному регистру. Удалили из таблицы все дубликаты

### Лемматизация

In [22]:
clients_data['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     767
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Вывел список уникальных целей кредита

С помощью модуля pymystem3 лемматизируем полученный выше список, получив  уникальные леммы:

In [23]:
m = Mystem()

list_of_lemmas = []

for element in clients_data['purpose']:
    lemma = m.lemmatize(element)
    list_of_lemmas.extend(lemma)

unique_lemmas = Counter(list_of_lemmas)

sorted(unique_lemmas.items(), key = lambda pair: pair[1], reverse=True)

[(' ', 33568),
 ('\n', 21453),
 ('недвижимость', 6351),
 ('покупка', 5897),
 ('жилье', 4460),
 ('автомобиль', 4306),
 ('образование', 4013),
 ('с', 2918),
 ('операция', 2604),
 ('свадьба', 2323),
 ('свой', 2230),
 ('на', 2221),
 ('строительство', 1878),
 ('высокий', 1374),
 ('получение', 1314),
 ('коммерческий', 1311),
 ('для', 1289),
 ('жилой', 1230),
 ('сделка', 941),
 ('дополнительный', 906),
 ('заниматься', 904),
 ('проведение', 767),
 ('сыграть', 765),
 ('сдача', 651),
 ('семья', 638),
 ('собственный', 635),
 ('со', 627),
 ('ремонт', 607),
 ('подержанный', 486),
 ('подержать', 478),
 ('приобретение', 461),
 ('профильный', 436)]

Среди всех возможных целей получения кредита можно выделить всего 5 ключевых слов, которые отражают главную цель.

In [24]:
categories = ["жилье", "образование", "свадьба", "недвижимость", "автомобиль"]

def lemmatize(text):
    lemma = m.lemmatize(text)
    for word in categories:
        if word in lemma:
            lemma = word
    return lemma

clients_data['purpose_group'] = clients_data['purpose'].apply(lemmatize)    

In [25]:
clients_data['purpose_group'].value_counts()

недвижимость    6351
жилье           4460
автомобиль      4306
образование     4013
свадьба         2323
Name: purpose_group, dtype: int64

Объединим недвижимость и жилье, т.к жилье тоже относится к недвижимости.

In [26]:
clients_data.loc[clients_data['purpose_group'] == 'жилье', 'purpose_group'] = 'недвижимость'
clients_data['purpose_group'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2323
Name: purpose_group, dtype: int64

**Вывод**

Подобное объединение ключевых слов из целей клиентов упрощает дальнейший анализ и сохраняет основной смысл цели получения кредита.

### Категоризация данных

In [27]:
def categories(i):
    if i == 'недвижимость':
        return 0
    elif i == 'автомобиль':
        return 1
    elif i == 'образование':
        return 2
    elif i == 'свадьба':
        return 3

In [28]:
clients_data.loc[:, 'categories_id'] = clients_data['purpose_group'].apply(categories)

In [29]:
clients_data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group,categories_id
0,1,8437,42.00,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,0
1,1,4024,36.00,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,1
2,0,5623,33.00,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,0
3,3,4124,32.00,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,2
4,0,14177,53.00,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,3
5,0,926,27.00,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость,0
6,0,2879,43.00,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость,0
7,0,152,50.00,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование,2
8,2,6929,35.00,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба,3
9,0,2188,41.00,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость,0


**Вывод**

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [30]:
clients_data.loc[clients_data['children'] == 20, 'children'] = 2 # Заменим ошибочные значения вместо 20 детей укажем 2-х,
clients_data.loc[clients_data['children'] == -1, 'children'] = 1 # вместо -1 укажем 1.

In [31]:
debt_from_children = pd.DataFrame()
debt_from_children['count_children'] = clients_data.groupby('children')['debt'].count()
debt_from_children['sum_children'] = clients_data.groupby('children')['debt'].sum()
debt_from_children['result_children'] = debt_from_children['sum_children'] / debt_from_children['count_children'] 
debt_from_children.sort_values('result_children', ascending = False)

,count_children,sum_children,result_children
children,,,
4,41,4,0.10
2,2128,202,0.09
1,4855,445,0.09
3,330,27,0.08
0,14090,1063,0.08
5,9,0,0.00


**Вывод**

С увеличением количества детей мы видим увеличение количества просроченных задолженностей, хотя люди с 3 детьми чаще платят в срок чем люди с 1 ребенком. Данные не однозначные, возможно, нужна большая выборка, чем та, которую мы имеем. Бездетные, как правило реже просрачивают оплату по кредиту, чем люди с детьми.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [32]:
debt_from_family_status = pd.DataFrame()
debt_from_family_status['sum_family_status'] = clients_data.groupby('family_status')['debt'].sum()
debt_from_family_status['count_family_status'] = clients_data.groupby('family_status')['debt'].count()
debt_from_family_status['result_family_status'] = debt_from_family_status['sum_family_status'] / debt_from_family_status['count_family_status'] 
debt_from_family_status.sort_values('result_family_status', ascending = False)

,sum_family_status,count_family_status,result_family_status
family_status,,,
Не женат / не замужем,274,2810,0.10
гражданский брак,388,4150,0.09
женат / замужем,931,12339,0.08
в разводе,85,1195,0.07
вдовец / вдова,63,959,0.07


**Вывод**

Да, зависимость есть. Люди не в браке и не бывавшие в браке имеют больший процент невозвратов в срок. Но, те кто развелись или овдовели чаще платят в срок, чем люди в браке.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [33]:
total_income = pd.qcut(clients_data['total_income'], 4)

income_pivot = clients_data.pivot_table(index=[total_income], aggfunc={'family_status_id':len, 'debt':sum})

income_pivot

,debt,family_status_id
total_income,,
"(20666.999, 107620.0]",427,5364
"(107620.0, 145017.0]",547,6414
"(145017.0, 195818.0]",384,4312
"(195818.0, 2265604.0]",383,5363


In [34]:
income_pivot['part'] = income_pivot['debt'] / income_pivot['family_status_id'] * 100

income_pivot.sort_values(by='part', ascending = False)

,debt,family_status_id,part
total_income,,,
"(145017.0, 195818.0]",384,4312,8.91
"(107620.0, 145017.0]",547,6414,8.53
"(20666.999, 107620.0]",427,5364,7.96
"(195818.0, 2265604.0]",383,5363,7.14


**Вывод**

Разбив квантилями столбец `total_income`, получим четыре примерно равных по численности группы. Наибольший процент должников - в группе чуть выше медианы, с доходом от медианы до примерно 200 000 руб/мес, а клиенты с наименьшими и наибольшими доходами реже остальных задерживают платежи по кредитам. Таким образом, уровень дохода практически не влияет на возврат кредита в срок. 

- Как разные цели кредита влияют на его возврат в срок?

In [35]:
debt_from_purpose_category = pd.DataFrame()
debt_from_purpose_category['sum_purpose_group'] = clients_data.groupby('purpose_group')['debt'].sum()
debt_from_purpose_category['count_purpose_group'] = clients_data.groupby('purpose_group')['debt'].count()
debt_from_purpose_category['result_purpose_group'] = debt_from_purpose_category['sum_purpose_group'] / debt_from_purpose_category['count_purpose_group'] 
debt_from_purpose_category.sort_values('result_purpose_group', ascending = False)

,sum_purpose_group,count_purpose_group,result_purpose_group
purpose_group,,,
автомобиль,403,4306,0.09
образование,370,4013,0.09
свадьба,186,2323,0.08
недвижимость,782,10811,0.07


**Вывод**

Как видно из анализа, среди 4 категорий, чаще всего платят в срок с целью кредита недвижимость. Кредиты на свадьбу просрочивают реже - примерно как и в случае недвижимости. Чаще просрочка у тех кому нужен кредит на авто и образование. 

## Шаг 4. Общий вывод

Проанализировав данные, видим следующее:

Процент должников среди родителей заметно выше, чем среди бездетных заемщиков: в случае рождения детей вероятность стать должником по кредиту возрастает.

Также есть явная зависимость между семейным положением и возвратом кредита в срок: неженатые / незамужние клиенты чаще всего становятся должниками. А вот семейные клиенты, вдовцы и в разводе - самые надежные плательщики.

Между уровнем дохода и возвратом кредита в срок зависимость минимальна.

Цель кредита влияет на вероятность просрочки платежей: если цель кредита - автомобиль или образование, то вероятность явно выше, чем если целью кредита являются операции с недвижимостью.